# M4L Demucs Worker (One Notebook)

Run all cells once. Drops-only workflow is default. Advanced users can flip `zero_config` in Drive config.


In [ ]:
# Cell 1: Mount Drive and show GPU (warn if CPU)
from google.colab import drive
import subprocess

drive.mount('/content/drive', force_remount=True)
smi = subprocess.getoutput('nvidia-smi')
print(smi)
if 'NVIDIA-SMI' not in smi:
    print('\nWARNING: GPU not detected. Go to Runtime > Change runtime type > Select GPU, then rerun.')


In [ ]:
# Cell 2: Fast bootstrap — wheelhouse + cached site-packages + model cache (verbose)
import os, sys, subprocess, pathlib, re, json, shutil, time
BASE = pathlib.Path('/content/drive/MyDrive/M4L-Demucs')
SITE = BASE / '.venv' / 'site-packages'
WHEELS = BASE / 'wheelhouse'
PIP_CACHE = BASE / 'pip-cache'
MODEL_CACHE = BASE / 'model-cache'
MARKER = BASE / '.env.json'
for p in [SITE, WHEELS, PIP_CACHE, MODEL_CACHE]: p.mkdir(parents=True, exist_ok=True)

os.environ.setdefault('XDG_CACHE_HOME', str(MODEL_CACHE))
os.environ.setdefault('DEMUCS_CACHE', str(MODEL_CACHE))

if str(SITE) not in sys.path:
    sys.path.insert(0, str(SITE))

print('Bootstrap start. wheelhouse=', WHEELS)

def detect_cuda_flavor():
    smi = subprocess.getoutput('nvidia-smi')
    m = re.search(r'CUDA Version:\s*([0-9.]+)', smi)
    if not m:
        raise RuntimeError('GPU not detected. Please set Runtime > Change runtime type > GPU and rerun.')
    ver = m.group(1)
    if ver.startswith('12.4'):
        return ['cu124','cu126','cu121','cu118']
    if ver.startswith('12.6'):
        return ['cu126','cu121','cu118']
    if ver.startswith('12.'):
        return ['cu126','cu121','cu118']
    if ver.startswith('11.8'):
        return ['cu118','cu121']
    return ['cu126','cu121','cu118']

# Torch/torchaudio versions known to exist per flavor
TORCH_MAP = {
    'cu124': ('2.5.1','2.5.1','https://download.pytorch.org/whl/cu124'),
    'cu126': ('2.8.0','2.8.0','https://download.pytorch.org/whl/cu126'),
    'cu121': ('2.4.0','2.4.0','https://download.pytorch.org/whl/cu121'),
    'cu118': ('2.1.2','2.1.2','https://download.pytorch.org/whl/cu118'),
}
DEMUCS_VER = '4.0.1'

# Load previous marker
cur = {}
if MARKER.exists():
    try: cur = json.loads(MARKER.read_text() or '{}')
    except Exception: cur = {}
print('Marker current:', cur)

# Try flavors in order (GPU only; no CPU fallback)
flavors = detect_cuda_flavor()
print('CUDA flavors preference:', flavors)
chosen = None
for fl in flavors:
    t0 = time.time()
    torch_ver, ta_ver, idx_url = TORCH_MAP[fl]
    want = { 'torch': f'{torch_ver}+{fl}', 'torchaudio': f'{ta_ver}+{fl}', 'demucs': DEMUCS_VER }
    need_refresh = (cur != want)
    print(f'Check flavor {fl}: want={want} need_refresh={need_refresh}')
    if need_refresh:
        print(f'Cleaning old wheels and downloading for {fl} …')
        for name in os.listdir(WHEELS):
            if name.startswith(('torch','torchaudio','demucs')) and name.endswith('.whl'):
                try: os.remove(WHEELS/name)
                except: pass
        cmd = ['pip','download','-d',str(WHEELS),'--prefer-binary','--only-binary=:all:','--extra-index-url',idx_url,
               f'torch=={torch_ver}+{fl}', f'torchaudio=={ta_ver}+{fl}', f'demucs=={DEMUCS_VER}', 'ffmpeg-python','numpy']
        print('DOWNLOAD cmd:', ' '.join(cmd))
        res = subprocess.run(cmd, text=True)
        if res.returncode != 0:
            print(f'Wheel download failed for {fl}. Attempting direct install to Drive site-packages…')
            direct = [sys.executable,'-m','pip','install','--no-warn-script-location','--extra-index-url',idx_url,
                      '--upgrade','--target',str(SITE), f'torch=={torch_ver}+{fl}', f'torchaudio=={ta_ver}+{fl}', f'demucs=={DEMUCS_VER}', 'ffmpeg-python','numpy']
            print('DIRECT INSTALL cmd:', ' '.join(direct))
            if subprocess.run(direct, text=True).returncode != 0:
                print(f'Direct install failed for {fl} (elapsed {time.time()-t0:.1f}s). Trying next flavor…')
                continue
            print('Direct install OK. Caching wheels (best effort)…')
            subprocess.run(['pip','download','-d',str(WHEELS),'--prefer-binary','--only-binary=:all:','--extra-index-url',idx_url,
                            f'torch=={torch_ver}+{fl}', f'torchaudio=={ta_ver}+{fl}', f'demucs=={DEMUCS_VER}', 'ffmpeg-python','numpy'], text=True)
        MARKER.write_text(json.dumps(want))
        print(f'Wrote marker: {want} (elapsed {time.time()-t0:.1f}s)')
        chosen = (fl, torch_ver, ta_ver)
        break
    else:
        print(f'Using cached marker for {fl}.')
        chosen = (fl, torch_ver, ta_ver)
        break

if not chosen:
    raise RuntimeError('Failed to setup torch/demucs for CUDA flavors (cu124/cu126/cu121/cu118). Try switching Colab GPU and rerun.')

fl, torch_ver, ta_ver = chosen
print('Installing from wheelhouse into site-packages… (', fl, ')')
install_cmd = [sys.executable,'-m','pip','install','--no-warn-script-location','--no-index','--find-links',str(WHEELS),
     '--upgrade','--target',str(SITE),'torch','torchaudio','demucs','ffmpeg-python','numpy']
print('INSTALL cmd:', ' '.join(install_cmd))
subprocess.run(install_cmd, text=True)
# Quiet fix for IPython warning
subprocess.run([sys.executable,'-m','pip','install','-q','--target',str(SITE),'jedi>=0.16'], text=True)

import demucs, torch  # noqa
print('Torch CUDA available:', torch.cuda.is_available(), '| torch cuda version:', torch.version.cuda)
if not torch.cuda.is_available():
    raise RuntimeError('Torch CUDA not available after install. Switch Colab runtime to GPU and rerun.')
print('Boot OK. demucs', demucs.__version__, '| torch', torch.__version__, 'cuda', torch.cuda.is_available(), '| flavor', fl, '| wheelhouse', WHEELS)
print('site-packages:', SITE)
print('model-cache:', MODEL_CACHE)


In [ ]:
# Cell 3: Config reader (zero_config default true)
import json, pathlib
ROOT = pathlib.Path('/content/drive/MyDrive/M4L-Demucs')
ROOT.mkdir(parents=True, exist_ok=True)
CONFIG = ROOT / 'config.json'
if not CONFIG.exists():
    CONFIG.write_text(json.dumps({"zero_config": True}))
print('Config:', CONFIG.read_text())


In [ ]:
# Cell 4: Watcher loop
import urllib.request, subprocess, sys
watcher_url = 'https://raw.githubusercontent.com/VSTOPIA/Doctor-Sample-Unit-DSU/main/colab_watcher.py'
urllib.request.urlretrieve(watcher_url, ROOT / 'colab_watcher.py')
print('Watcher fetched')
!python /content/drive/MyDrive/M4L-Demucs/colab_watcher.py


In [ ]:
# Optional: one-shot run for a specific file (debug)
# Place a file at /content/drive/MyDrive/M4L-Demucs/jobs/audio/YourSong.wav
# Then uncomment and run below:
# import subprocess, sys, pathlib
# IN_WAV = pathlib.Path('/content/drive/MyDrive/M4L-Demucs/jobs/audio/YourSong.wav')
# OUT = pathlib.Path('/content/drive/MyDrive/M4L-Demucs/out/oneshot')
# OUT.mkdir(parents=True, exist_ok=True)
# cmd = [sys.executable,'-m','demucs.separate','-n','htdemucs_ft','-o',str(OUT),'-j','4','--shifts','4','--two-stems','vocals',str(IN_WAV)]
# print(' '.join(cmd))
# subprocess.run(cmd)


In [ ]:
# Diagnostics: confirm GPU + cached installs
import sys, subprocess, pathlib, json
BASE = pathlib.Path('/content/drive/MyDrive/M4L-Demucs')
SITE = BASE/'.venv'/'site-packages'
WHEELS = BASE/'wheelhouse'
MODELS = BASE/'model-cache'
print('python:', sys.version)
print('nvidia-smi:')
print(subprocess.getoutput('nvidia-smi'))
try:
  import torch, demucs
  print('torch:', torch.__version__, 'cuda avail:', torch.cuda.is_available(), 'torch cuda:', torch.version.cuda)
  print('demucs:', demucs.__version__)
  print('demucs path:', getattr(demucs,'__file__','?'))
  print('torch path:', getattr(torch,'__file__','?'))
except Exception as e:
  print('Import error:', e)
print('site-packages exists:', SITE.exists(), 'wheelhouse exists:', WHEELS.exists(), 'models exists:', MODELS.exists())
